ADAPTIVE PERSUASION SYSTEM - PRODUCTION VERSION

In [6]:
!pip install -q transformers>=4.40.0 huggingface_hub>=0.23.0 torch
!pip install -q textblob nltk pandas numpy matplotlib

import torch, transformers, textblob, nltk, pandas, numpy, matplotlib
print("All required libraries are installed and imported successfully.")


All required libraries are installed and imported successfully.


In [7]:
# Imports

import os
import json
import re
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import random

from huggingface_hub import InferenceClient, login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from textblob import TextBlob
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML


# NLTK Downloads
nltk.download('punkt', quiet=True)
nltk.download('vader_lexicon', quiet=True)


True

In [8]:
# Authentication

HF_TOKEN = os.getenv("HF_TOKEN")
os.environ['HF_TOKEN'] = HF_TOKEN

try:
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Authenticated\n")
except Exception as e:
    print(f"Failed: {e}")
    raise

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Authenticated



In [9]:
# Loading the llama 3.1 model

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    print("Model loaded successfully!\n")
    USE_LOCAL_MODEL = True
except Exception as e:
    print(f"Could not load model locally: {e}")
    print("Falling back to Inference API until access is approved\n")
    USE_LOCAL_MODEL = False
    client = InferenceClient(api_key=HF_TOKEN)


Could not load model locally: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-696264d4-64e04ebe6122323f382652f4;cbdba31c-ed6c-4434-822a-b0e65e5d713e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Llama-3.1-8B-Instruct is awaiting a review from the repo authors.
Falling back to Inference API until access is approved



In [10]:
# CAMPAIGN SETUP

org_name = input("Organization [Children's Education Fund]: ").strip() or "Children's Education Fund"
cause = input("Cause [education for children]: ").strip() or "providing education to underprivileged children"
amounts = input("Amounts [200,500,1000]: ").strip() or "200, 500, 1000"
impact = input("Impact [₹200 = 5 kids supplies]: ").strip() or "₹200 provides school supplies for 5 children for a month"

DONATION_CONTEXT = {
    'organization': org_name,
    'cause': cause,
    'amounts': amounts,
    'impact': impact
}

print("\nCampaign ready\n")


Campaign ready



In [11]:
# CONFIGURATION

class Config:
    MODEL_NAME = MODEL_NAME
    TEMPERATURE = 0.8
    MAX_NEW_TOKENS = 64

    INITIAL_BELIEF = 0.05
    INITIAL_TRUST = 1.0
    TRUST_THRESHOLD = 0.5

    ALPHA = 0.1
    BETA = 0.15
    GAMMA = 0.08

    HARD_REJECTION_PENALTY = 0.4
    SOFT_REJECTION_PENALTY = 0.2
    MIN_STRATEGY_WEIGHT = 0.05

    MAX_TURNS = 15
    MAX_CONSECUTIVE_REJECTIONS = 3

    LOG_FILE = "dialogue_log.jsonl"
    STRATEGIES = ["Empathy", "Impact", "SocialProof", "Transparency", "EthicalUrgency"]

In [15]:
# REJECTION DETECTOR

class RejectionDetector:

    # EXPLICIT NO - very clear rejection
    EXPLICIT_PATTERNS = [
        r'\b(no thanks|no thank you|not interested|don\'t want|won\'t donate)\b',
        r'\b(nope|nah|never|absolutely not|definitely not)\b',
        r'\b(leave me alone|stop asking|not doing|refuse)\b'
    ]

    # SOFT NO - hesitation, delay
    SOFT_PATTERNS = [
        r'\b(maybe later|not now|not right now|some other time|another time)\b',
        r'\b(not today|can\'t right now|busy right now)\b',
        r'\b(not sure|unsure|uncertain|hesitant|doubtful)\b',
        r'\b(can\'t afford|no money|tight budget|broke|expensive)\b',
        r'\b(i\'ll think|let me think|need time|consider)\b'
    ]

    # TRUST ISSUES
    TRUST_PATTERNS = [
        r'\b(pushy|aggressive|pressure|uncomfortable|sketchy|scam|fraud)\b',
        r'\b(suspicious|don\'t trust|seems fake|sounds fake)\b',
        r'\b(why are you|what\'s your motive|prove it)\b'
    ]

    # CURIOSITY - asking questions, wanting info
    CURIOSITY_PATTERNS = [
        r'\b(tell me more|tell me about|what about|explain|how does|how do)\b',
        r'\b(more info|more details|details|information)\b',
        r'\b(curious|interested in learning|want to know|want to hear)\b',
        r'\b(what is|who are|where does|when|why)\b',
        r'\b(can you|could you|would you.*explain|show me)\b'
    ]

    # ACTUAL COMMITMENT - very explicit intent to donate
    ACCEPTANCE_PATTERNS = [
        r'\b(yes.*i.*donate|i will donate|i\'ll donate|i want to donate)\b',
        r'\b(sign me up|count me in|i\'m in)\b',
        r'\b(where do i donate|how do i donate|how can i donate)\b',
        r'\b(i\'ll give|i will give|i want to give ₹|i can give ₹)\b',
        r'\b(okay.*donate|ok.*donate|sure.*donate|let\'s do it)\b',
        r'\b(take my donation|here\'s my donation|ready to donate)\b'
    ]


    def __init__(self):
        self.sentiment = None

    def detect(self, user_message: str) -> Dict:
        msg = user_message.lower().strip()

        # First check: ACTUAL donation commitment
        if self._match(msg, self.ACCEPTANCE_PATTERNS):
            return {
                'rejection_type': 'none',
                'rejection_confidence': 0.0,
                'trust_concern': False,
                'sentiment_score': 0.9,
                'sentiment_label': 'positive',
                'is_acceptance': True,
                'is_curiosity': False
            }

        # Check curiosity
        is_curiosity = self._match(msg, self.CURIOSITY_PATTERNS)

        # Check rejection
        rejection_type = 'none'
        confidence = 0.0

        if self._match(msg, self.EXPLICIT_PATTERNS):
            rejection_type = 'explicit'
            confidence = 0.9
        elif self._match(msg, self.SOFT_PATTERNS):
            rejection_type = 'soft'
            confidence = 0.7

        # Trust concerns
        trust_concern = self._match(msg, self.TRUST_PATTERNS)

        # Sentiment
        sent_score, sent_label = self._get_sentiment(user_message)

        # Special handling
        if is_curiosity and rejection_type == 'none':
            rejection_type = 'curiosity'
            sent_score = max(0.2, sent_score)

        if sent_score < -0.4 and rejection_type == 'none':
            rejection_type = 'ambiguous'
            confidence = 0.5

        if sent_score < -0.6 and rejection_type == 'soft':
            rejection_type = 'explicit'
            confidence = 0.85

        return {
            'rejection_type': rejection_type,
            'rejection_confidence': confidence,
            'trust_concern': trust_concern,
            'sentiment_score': sent_score,
            'sentiment_label': sent_label,
            'is_acceptance': False,
            'is_curiosity': is_curiosity
        }


    def _match(self, text: str, patterns: List[str]) -> bool:
        return any(re.search(p, text, re.IGNORECASE) for p in patterns)

    def _get_sentiment(self, text: str) -> Tuple[float, str]:
        try:
            if self.sentiment:
                r = self.sentiment(text[:512])[0]
                score = r['score'] if r['label'] == 'POSITIVE' else -r['score']
                return score, r['label'].lower()
        except:
            pass

        blob = TextBlob(text)
        pol = blob.sentiment.polarity
        label = 'positive' if pol > 0 else ('negative' if pol < 0 else 'neutral')
        return pol, label
